In [1]:
from sympy import symbols, expand,sympify,nsimplify
import pandas as pd

In [2]:
def create_newExpr(file_num):
    filePath = "./data/A_raw/"+file_num+"A.lg"
    # print(filePath)
    with open(filePath,'r',encoding='utf-8') as f:
        text = f.read()

    expressions = [exp.strip() for exp in text.split('&') if exp.strip()]

    rules = {}
    for exp in expressions:
        # Split the expression into left-hand side and right-hand side of '='
        if '=' in exp:
            parts = exp.split('=', 1)
            name = parts[0].strip()
            definition = parts[1].strip()
            
            # Remove trailing '&' if it is a pure function
            if definition.endswith('&'):
                definition = definition[:-1].strip()
            
            try:
                # Convert to a SymPy expression
                # First replace Mathematica-style #1/#2 with Python-recognizable variables
                definition = definition.replace('#1', 'x').replace('#2', 'y')
                expr = sympify(definition)
                
                rules[name] = expr
            except Exception as e:
                print(f"wrong: {definition}, error: {e}")

    keys = list(rules.keys())
    values = list(rules.values())

    newExprs = []
    for i in range(len(keys)):
        t0, t1, x, y = symbols('t0 t1 x y')
        expr = values[i].subs({
            x: t0**(-1/2) * t1**(-1/2),
            y: t1**(-1/4) / t0**(-1/4)
        }) # replace (x,y) with (t0^{-1/2}*t1^{-1/2}, t1^{-1/4}/t0^{-1/4}) 
        newExpr = expand(expr)
        newExpr = nsimplify(newExpr, rational=True)
        newExprs.append(newExpr)
        # break
    D = pd.DataFrame(columns=['key','original','newExpr'])
    D['key'] = keys
    D['original'] = values
    D['newExpr'] = newExprs

    D.to_csv('./data/A_new/'+'new_'+file_num+'A.csv',index=False)

In [3]:
# Testing number 05 
create_newExpr('05')

In [4]:
lists = [f"{i:02d}" for i in range(3, 17)]

In [5]:
lists

['03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16']

In [6]:
# # Testing all numbers from 03 to 16. Note that testing from 12 to 16 need some time.
# for num in lists:
#     create_newExpr(num)
#     print('OK for ', num)
# print('Done')